<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span><ul class="toc-item"><li><span><a href="#Singleton-vs-Dependency-Injection" data-toc-modified-id="Singleton-vs-Dependency-Injection-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Singleton vs Dependency Injection</a></span></li></ul></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span><ul class="toc-item"><li><span><a href="#Lazy-Init" data-toc-modified-id="Lazy-Init-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Lazy Init</a></span></li><li><span><a href="#Synchronised" data-toc-modified-id="Synchronised-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Synchronised</a></span></li><li><span><a href="#Double-checked-locking" data-toc-modified-id="Double-checked-locking-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Double checked locking</a></span></li><li><span><a href="#Eager-init" data-toc-modified-id="Eager-init-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Eager init</a></span></li><li><span><a href="#Bill-Pugh---Best-way-for-Java" data-toc-modified-id="Bill-Pugh---Best-way-for-Java-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Bill Pugh - Best way for Java</a></span></li></ul></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Singleton

## Intro

The singleton pattern is a pretty simple patter. It's purpose is to ensure that only a *single* instance of a class exists and that if it is referred to again, that only that instance is returned.

The class itself manages this behaviour and the only way to access the instance is via the class.

A global access point is needed also. This again will be on the class itself.

A good example of a singleton would be a window manager or a cache. Only one of these things should ever exist at a time.

Some advantages to this pattern are:
* Controlled access to a sole instance (strict control over how it's accessed)
* Reduced name space - avoids polluting name space with global variables
* Permits a variable number of instances - a singleton is configurable. We like configurable.

Some disadvantages to the pattern are:
* Hindering unit testing (basically need the singleton in test which can cause issues depending on how you test - mocks can solve this)
* Can create hidden dependencies. It's reference is obscured as part of the pattern, causing it to be baked into things that reference it.

The pattern works by having a method that is used to instantiate the object in question. No public constructor calls here!

When the method is called, it checks to see if an instance exists. If it doesn't it creates one. If it does, it returns the existing instance.

### Singleton vs Dependency Injection

Singletons and Dependency injection can be somewhat similar. They both use specific constructor behaviour to achieve management of class instances and they help to manage cognitive load with methods.

Dependency injection avoids the use of static methods (the dependency is only injected at runtime/ instance creation). This is one of the most common reasons we should choose dependency injection.

DI often uses the constructor to create a single object. Whereas, the singleton pattern abuses constructor behaviour and scoping rules with a static method. This means that the class doesn't need to be built ever, making it a bit difficult to track the dependency if one actually exists.

The way to choose which to use relies on how the instance will be used. If the instance is responsible for something independent from most things in the system (window manager) a singleton is best.

Otherwise, dependency injection is the better choice. as it will allow for stronger context boundaries and easier testing (email client).

Basically think of how many layers it spans. If it is only native to a particular layer (repository), it should probably use DI. If it is something that is likely to be used in multiple places, but not actually factor into the mechanics at all, it should probably be a singleton.

## Implementation

### Lazy Init 

In [3]:
class Singleton {
    private static Singleton uniqueInstance;
    
    private Singleton(){}
    
    public static Singleton getInstance()
    {
        if(uniqueInstance == null){
            uniqueInstance = new Singleton();
        }
        
        return uniqueInstance;
    }
}

The most basic way to implement the singleton pattern. This is not at all ideal. It is not thread-safe and doesn't really scale all that well.

Advantages:
* Simple
* Easy to read

Disadvantages:
* Not thread-safe

In [7]:
Singleton mySingleton = Singleton.getInstance();
System.out.println(mySingleton);

Singleton mySingletonTwo = Singleton.getInstance();
System.out.println(mySingletonTwo);

REPL.$JShell$14$Singleton@129f99f6
REPL.$JShell$14$Singleton@129f99f6


### Synchronised

In [ ]:
class Singleton {
    private static Singleton uniqueInstance;
    
    private Singleton(){}
    
    public static synchronized Singleton getInstance()
    {
        if(uniqueInstance == null){
            uniqueInstance = new Singleton();
        }
        
        return uniqueInstance;
    }
}

In [10]:
Singleton mySingleton = Singleton.getInstance();
System.out.println(mySingleton);

Singleton mySingletonTwo = Singleton.getInstance();
System.out.println(mySingletonTwo);

REPL.$JShell$14C$Singleton@7a4d1c0f
REPL.$JShell$14C$Singleton@7a4d1c0f


Effective and straightforward. Still not the best possible way in Java, but this is a decent analogue for most other languages to implement. If the synchronized key word doesn't exist in the lang, find the way to create blocking ops (most languages have this or a GIL).

Advantages:
* Thread-safe
* Straight forward and effective

Disadvantages:
* Not performance focused

### Double checked locking

In [11]:
class Singleton {
    private volatile static Singleton uniqueInstance;
    
    private Singleton(){}
    
    public static Singleton getInstance()
    {
        if(uniqueInstance == null){
            synchronized (Singleton.class){
                if(uniqueInstance == null){
                    uniqueInstance = new Singleton();
                }
            }
        }
        
        return uniqueInstance;
    }
}

In [12]:
Singleton mySingleton = Singleton.getInstance();
System.out.println(mySingleton);

Singleton mySingletonTwo = Singleton.getInstance();
System.out.println(mySingletonTwo);

REPL.$JShell$14E$Singleton@48d06287
REPL.$JShell$14E$Singleton@48d06287


Much better way of implementing the singleton. Has better performance and is also thread-safe. Not really an analogue to use in other languages. Is also a bit messy with the nested ifs, but it is an option and one that likely is used in the wild.

### Eager init

In [13]:
class Singleton {
    private static Singleton uniqueInstance = new Singleton();
    
    private Singleton(){}
    
    public static Singleton getInstance()
    {        
        return uniqueInstance;
    }
}

In [14]:
Singleton mySingleton = Singleton.getInstance();
System.out.println(mySingleton);

Singleton mySingletonTwo = Singleton.getInstance();
System.out.println(mySingletonTwo);

REPL.$JShell$14G$Singleton@53fc5dca
REPL.$JShell$14G$Singleton@53fc5dca


This is one of the best ways to implement the singleton pattern in Java. It is even better from a performance perspective than the other implementations and doesn't use many resources. But, it does mean that the singleton will always be created.

Advantages:
* thread-safe
* doesn't use many resources

Disadvantages:
* Will always create an instance of the singleton

### Bill Pugh - Best way for Java

In [15]:
class Singleton {

    private static class SingletonHelper {
        private static final Singleton uniqueInstance = 
                                                    new Singleton();
    }
    
    private Singleton(){}
    
    public static Singleton getInstance()
    {
        return SingletonHelper.uniqueInstance;
    }
}

In [16]:
Singleton mySingleton = Singleton.getInstance();
System.out.println(mySingleton);

Singleton mySingletonTwo = Singleton.getInstance();
System.out.println(mySingletonTwo);

REPL.$JShell$14I$Singleton@b675c80
REPL.$JShell$14I$Singleton@b675c80


This approach is pod racing. This uses lots of cool, but basic Java features to ensure that the instance is returned. The final keyword ensures that the variable cannot be reassigned. 

The nested inner class ensures that the encapsulated variable is not instantiated until it is called by the getInstance method.

Because of the above, this is the best possible implementation in Java.

## Example

In [19]:
class CricketTeam {
    
    private String captainName;

    private static class TeamHelper {
        private static final CricketTeam cricketTeam = new CricketTeam();
    }
    
    private CricketTeam(){}
    
    public static CricketTeam getCricketTeam()
    {
        return TeamHelper.cricketTeam;
    }
    
    public void setCaptain(String captainName)
    {
        this.captainName = captainName;
    }
    
    public void printCaptain()
    {
        System.out.println(this.captainName);
    }
}

In [22]:
CricketTeam myTeam = CricketTeam.getCricketTeam();
myTeam.setCaptain("Edward");
myTeam.printCaptain();

Edward


In [24]:
CricketTeam myTeamTwo = CricketTeam.getCricketTeam();
myTeam.printCaptain();

Edward


In [26]:
myTeam == myTeamTwo;

true